In [1]:
import org.apache.spark.SparkContext
import org.apache.spark.SparkConf
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD
import org.apache.spark.mllib.regression.LabeledPoint
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import org.apache.spark.mllib.tree.DecisionTree
import org.apache.spark.mllib.tree.RandomForest
import org.apache.spark.mllib.tree.GradientBoostedTrees
import org.apache.spark.mllib.tree.configuration.BoostingStrategy
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics
import java.nio.file.{Paths, Files}
import java.nio.charset.StandardCharsets
import scala.util.control.Breaks._
import scala.math._

Intitializing Scala interpreter ...

Spark Web UI available at http://128.232.240.137:4040
SparkContext available as 'sc' (version = 3.1.2, master = local[*], app id = local-1642502821365)
SparkSession available as 'spark'


import org.apache.spark.SparkContext
import org.apache.spark.SparkConf
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD
import org.apache.spark.mllib.regression.LabeledPoint
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import org.apache.spark.mllib.tree.DecisionTree
import org.apache.spark.mllib.tree.RandomForest
import org.apache.spark.mllib.tree.GradientBoostedTrees
import org.apache.spark.mllib.tree.configuration.BoostingStrategy
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics
import java.nio.file.{Paths, Files}
import java.nio.charset.StandardCharsets
import scala.util.control.Breaks._
import scala.math._


In [2]:
val jobName = "siteRequest"
val conf = new SparkConf().setAppName(jobName).setMaster("local[*]").set("spark.executor.memory","10g").set("spark.driver.memory","10g")
val sc =  SparkContext.getOrCreate(conf)
conf.set("spark.streaming.stopGracefullyOnShutdown","true")

jobName: String = siteRequest
conf: org.apache.spark.SparkConf = org.apache.spark.SparkConf@7c073a91
sc: org.apache.spark.SparkContext = org.apache.spark.SparkContext@157101dc
res0: org.apache.spark.SparkConf = org.apache.spark.SparkConf@7c073a91


In [3]:
val RDData = sc.textFile("com-dblp.ungraph.txt");
val RDData2 = sc.textFile("unconnected.txt");
val RDData3 = sc.textFile("2hop_final.txt");
val Splitted1RDD = RDData.map(_.split("\t"));
val Splitted2RDD = RDData2.map(_.split(" "));
val Splitted3RDD = RDData3.map(_.split(" "));
val EdgeRDD = Splitted1RDD.map(row=>(row(0).toLong,row(1).toLong));
val Vertex1RDD = EdgeRDD.map{t => t._1}.distinct();
val Vertex2RDD = EdgeRDD.map{t => t._2}.distinct();
val VertexRDD = Vertex1RDD.union(Vertex2RDD).distinct();


val UnConnectedEdgeRDD = Splitted2RDD.map(row=>(row(0).toLong,row(1).toLong));
val twoHopEdgeRDD = Splitted3RDD.map(row=>(row(0).toLong,row(1).toLong));
// EdgeRDD.take(2).foreach(arr => println(arr.toList))
// UnConnectedEdgeRDD.take(2).foreach(println)

RDData: org.apache.spark.rdd.RDD[String] = com-dblp.ungraph.txt MapPartitionsRDD[1] at textFile at <console>:47
RDData2: org.apache.spark.rdd.RDD[String] = unconnected.txt MapPartitionsRDD[3] at textFile at <console>:48
RDData3: org.apache.spark.rdd.RDD[String] = 2hop_final.txt MapPartitionsRDD[5] at textFile at <console>:49
Splitted1RDD: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[6] at map at <console>:50
Splitted2RDD: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[7] at map at <console>:51
Splitted3RDD: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[8] at map at <console>:52
EdgeRDD: org.apache.spark.rdd.RDD[(Long, Long)] = MapPartitionsRDD[9] at map at <console>:53
Vertex1RDD: org.apache.spark.rdd.RDD[Long] = MapPartitionsRDD[13] at distinct at...


In [4]:
// println("Total Number of vertex: " + VertexRDD.count())
// println("Total Number of Connected Edge: " + EdgeRDD.count())
// println("Total Number of Unconnected Edge: " + UnConnectedEdgeRDD.count())
// println("Total Number of TwoHop Edge: " + twoHopEdgeRDD.count())

In [5]:
val deltaRDD = twoHopEdgeRDD.subtract(EdgeRDD)
val interRDD = twoHopEdgeRDD.intersection(EdgeRDD)
val deltaSmallRDD = sc.parallelize(deltaRDD.sample(false,0.1,0).collect())

deltaRDD: org.apache.spark.rdd.RDD[(Long, Long)] = MapPartitionsRDD[27] at subtract at <console>:50
interRDD: org.apache.spark.rdd.RDD[(Long, Long)] = MapPartitionsRDD[33] at intersection at <console>:51
deltaSmallRDD: org.apache.spark.rdd.RDD[(Long, Long)] = ParallelCollectionRDD[35] at parallelize at <console>:52


In [6]:
// deltaRDD.count()
// interRDD.count()
// deltaSmallRDD.count()

In [7]:
// Validation Graph = unConnectedEdge + connectedEdge
// Test Graph = (unConnectedEdge + 0.1*connectedEdge) + 0.9*connectedEdge
// Train Graph = ((unConnectedEdge + 0.1*connectedEdge) + 0.1*0.9*connectedEdge) + 0.9*0.9*connectedEdge

// What we want to predict for testgraph is 0.1*connectedEdge
// What we want to predict for traingraph is 0.1*0.9*connectedEdge

// test set is unConnectedEdge (label 0) + 0.1*connectedEdge (label 1)
// train set is unConnectedEdge + 0.1*connectedEdge (label 0) + 0.1*0.9*connectedEdge (label 1)

In [8]:
//Construct Test Set with Validation Graph and Testing Graph
//Missing Link for test graph: aRDD
val Array(aRDD, bRDD) = interRDD.randomSplit(Array(0.12, 0.88));
// TestSet positive: testSetPos
// val testSetPos = sc.parallelize(aRDD.map{t=>(t._1,t._2,1)}.sample(false,0.8,0).collect());
val testSetPos = aRDD.map{t=>(t._1,t._2,1)};
//TestSet negative: testSetNeg
var testSetNeg = sc.parallelize(deltaSmallRDD.map{t=>(t._1,t._2,0)}.sample(false,0.14,0).collect());
// var testSetNeg = UnConnectedEdgeRDD.map{t=>(t._1,t._2,0)};
var testSet = testSetPos.union(testSetNeg);
//Construct Train Set with Testing Graph and Training Graph
//Missing Link for train graph: cRDD
val Array(cRDD, dRDD) = bRDD.randomSplit(Array(0.25, 0.75));
// val trainSetPos = sc.parallelize(cRDD.map{t=>(t._1,t._2,1)}.sample(false,0.7,0).collect());
val trainSetPos = cRDD.map{t=>(t._1,t._2,1)};
val trainSetNeg = sc.parallelize(aRDD.union(deltaSmallRDD).map{t=>(t._1,t._2,0)}.sample(false,0.28,0).collect());
var trainSet = trainSetPos.union(trainSetNeg);

aRDD: org.apache.spark.rdd.RDD[(Long, Long)] = MapPartitionsRDD[36] at randomSplit at <console>:52
bRDD: org.apache.spark.rdd.RDD[(Long, Long)] = MapPartitionsRDD[37] at randomSplit at <console>:52
testSetPos: org.apache.spark.rdd.RDD[(Long, Long, Int)] = MapPartitionsRDD[38] at map at <console>:55
testSetNeg: org.apache.spark.rdd.RDD[(Long, Long, Int)] = ParallelCollectionRDD[41] at parallelize at <console>:57
testSet: org.apache.spark.rdd.RDD[(Long, Long, Int)] = UnionRDD[42] at union at <console>:59
cRDD: org.apache.spark.rdd.RDD[(Long, Long)] = MapPartitionsRDD[43] at randomSplit at <console>:62
dRDD: org.apache.spark.rdd.RDD[(Long, Long)] = MapPartitionsRDD[44] at randomSplit at <console>:62
trainSetPos: org.apache.spark.rdd.RDD[(Long, Long, Int)] = MapPartitionsRDD[45] at map at <...


In [9]:
// println(testSetPos.count())
// println(testSetNeg.count())
// println(testSet.count())
// println(trainSetPos.count)
// println(trainSetNeg.count)
// println(trainSet.count)

In [10]:
//Build similarity indices for each edge in testSet
//Build Test Graph
val GraphVertexRDD = VertexRDD.map(line=>(line,line)).distinct();
val nAllVertex = VertexRDD.count();
val TestGraphEdgeRDD = bRDD.map{t=>(Edge(t._1,t._2,1))}.distinct()
val testGraph = Graph(GraphVertexRDD,TestGraphEdgeRDD);
val testNeigh = testGraph.collectNeighborIds(EdgeDirection.Either)
val testBroadcastVar = sc.broadcast(testNeigh.collect())
// val testNValues = testBroadcastVar.value.toMap
val testCCBroadcastVar = sc.broadcast(testGraph.connectedComponents.vertices.collect().toMap)


val testFeature = testSet.mapPartitions(rows=>{
    val testNValues = testBroadcastVar.value.toMap
    val ccTest = testCCBroadcastVar.value
    rows.map(t=>{
        val n1_neigh = testNValues(t._1)
        val n2_neigh = testNValues(t._2)
        val commonNei = n1_neigh.intersect(n2_neigh)
        //Common neigbors
        val CN = commonNei.length
        //Jaccard’s Coefficient 
        val JC = n1_neigh.intersect(n2_neigh).size/n1_neigh.union(n2_neigh).size.toDouble
        //Adamic/Adar
        var AA = 0.0
        commonNei.foreach(x => AA += 1/Math.log10(testNValues(x).length))
        //Resource Allocation
        var RA = 0.0
        commonNei.foreach(x => RA += 1/testNValues(x).length.toDouble)
        //Preferential Attachment
        val PA = n1_neigh.length*n2_neigh.length
        //Adjusted Rand
        val a = commonNei.length
        val b = n1_neigh.toSet.diff(n2_neigh.toSet).size
        val c = n2_neigh.toSet.diff(n1_neigh.toSet).size
        val d = nAllVertex - n1_neigh.union(n2_neigh).length
        val AR = 2*(a*d-b*c)/((a+b)*(b+d)+(a+c)*(c+d)).toDouble
        //Neighborhood Distance
        val nd = commonNei.length / Math.sqrt(n1_neigh.length*n2_neigh.length).toDouble
        val ND = if (nd.isNaN) 0 else nd
        //Total Neighbours
        val TN = n1_neigh.union(n2_neigh).size
        //Node Degree U
        val UND = n1_neigh.length
        //Node Degree V
        val VND = n2_neigh.length
        //Same Community
        val SC = if (ccTest(t._1)==ccTest(t._2)) 1 else 0
        (t._1,t._2,t._3,CN,JC,AA,RA,PA,AR,ND,TN,UND,VND,SC)
    })
})

GraphVertexRDD: org.apache.spark.rdd.RDD[(Long, Long)] = MapPartitionsRDD[54] at distinct at <console>:54
nAllVertex: Long = 317080
TestGraphEdgeRDD: org.apache.spark.rdd.RDD[org.apache.spark.graphx.Edge[Int]] = MapPartitionsRDD[58] at distinct at <console>:56
testGraph: org.apache.spark.graphx.Graph[Long,Int] = org.apache.spark.graphx.impl.GraphImpl@7fd410de
testNeigh: org.apache.spark.graphx.VertexRDD[Array[org.apache.spark.graphx.VertexId]] = VertexRDDImpl[76] at RDD at VertexRDD.scala:57
testBroadcastVar: org.apache.spark.broadcast.Broadcast[Array[(org.apache.spark.graphx.VertexId, Array[org.apache.spark.graphx.VertexId])]] = Broadcast(20)
testCCBroadcastVar: org.apache.spark.broadcast.Broadcast[scala.collection.immutable.Map[org.apache.spark.graphx.VertexId,org.apache.spark.graphx....


In [11]:
//Build similarity indices for each edge in trainset
//Build Train Graph
val GraphVertexRDD = VertexRDD.map(line=>(line,line)).distinct();
val nAllVertex = VertexRDD.count();
val TrainGraphEdgeRDD = dRDD.map{t=>(Edge(t._1,t._2,1))}.distinct()
val trainGraph = Graph(GraphVertexRDD,TrainGraphEdgeRDD);
val trainNeigh = trainGraph.collectNeighborIds(EdgeDirection.Either)
val trainBroadcastVar = sc.broadcast(trainNeigh.collect())
// val trainNValues = trainBroadcastVar.value.toMap
val trainCCBroadcastVar = sc.broadcast(trainGraph.connectedComponents.vertices.collect().toMap)



val trainFeature = trainSet.mapPartitions(rows=>{
    val trainNValues = trainBroadcastVar.value.toMap
    val ccTrain = trainCCBroadcastVar.value
    rows.map(t=>{
        val n1_neigh = trainNValues(t._1)
        val n2_neigh = trainNValues(t._2)
        val commonNei = n1_neigh.intersect(n2_neigh)
        //Common neigbors
        val CN = commonNei.length
        //Jaccard’s Coefficient 
        val JC = n1_neigh.intersect(n2_neigh).size/n1_neigh.union(n2_neigh).size.toDouble
        //Adamic/Adar
        var AA = 0.0
        commonNei.foreach(x => AA += 1/Math.log10(trainNValues(x).length))
        //Resource Allocation
        var RA = 0.0
        commonNei.foreach(x => RA += 1/trainNValues(x).length.toDouble)
        //Preferential Attachment
        val PA = n1_neigh.length*n2_neigh.length
        //Adjusted Rand
        val a = commonNei.length
        val b = n1_neigh.toSet.diff(n2_neigh.toSet).size
        val c = n2_neigh.toSet.diff(n1_neigh.toSet).size
        val d = nAllVertex - n1_neigh.union(n2_neigh).length
        val AR = 2*(a*d-b*c)/((a+b)*(b+d)+(a+c)*(c+d)).toDouble
        //Neighborhood Distance
        val nd = commonNei.length / Math.sqrt(n1_neigh.length*n2_neigh.length).toDouble
        val ND = if (nd.isNaN) 0 else nd
        //Total Neighbours
        val TN = n1_neigh.union(n2_neigh).size
        //Node Degree U
        val UND = n1_neigh.length
        //Node Degree V
        val VND = n2_neigh.length
        //Same Community
        val SC = if (ccTrain(t._1)==ccTrain(t._2)) 1 else 0
        (t._1,t._2,t._3,CN,JC,AA,RA,PA,AR,ND,TN,UND,VND,SC)
    })
})

GraphVertexRDD: org.apache.spark.rdd.RDD[(Long, Long)] = MapPartitionsRDD[408] at distinct at <console>:57
nAllVertex: Long = 317080
TrainGraphEdgeRDD: org.apache.spark.rdd.RDD[org.apache.spark.graphx.Edge[Int]] = MapPartitionsRDD[412] at distinct at <console>:59
trainGraph: org.apache.spark.graphx.Graph[Long,Int] = org.apache.spark.graphx.impl.GraphImpl@67b6021e
trainNeigh: org.apache.spark.graphx.VertexRDD[Array[org.apache.spark.graphx.VertexId]] = VertexRDDImpl[430] at RDD at VertexRDD.scala:57
trainBroadcastVar: org.apache.spark.broadcast.Broadcast[Array[(org.apache.spark.graphx.VertexId, Array[org.apache.spark.graphx.VertexId])]] = Broadcast(106)
trainCCBroadcastVar: org.apache.spark.broadcast.Broadcast[scala.collection.immutable.Map[org.apache.spark.graphx.VertexId,org.apache.spar...


In [12]:
val parsedTestData = testFeature.map{x => 
    val parts1 = Array(x._1,x._2,x._4,x._5,x._6,x._7,x._8,x._9,x._10,x._11,x._12,x._13,x._14)
    LabeledPoint(x._3.toDouble, Vectors.dense(parts1.map(_.toDouble)))
}.cache();
val parsedTrainData = trainFeature.map{x => 
    val parts1 = Array(x._1,x._2,x._4, x._5,x._6,x._7,x._8,x._9,x._10,x._11,x._12,x._13,x._14)
    LabeledPoint(x._3.toDouble, Vectors.dense(parts1.map(_.toDouble)))
}.cache();

parsedTestData: org.apache.spark.rdd.RDD[org.apache.spark.mllib.regression.LabeledPoint] = MapPartitionsRDD[827] at map at <console>:49
parsedTrainData: org.apache.spark.rdd.RDD[org.apache.spark.mllib.regression.LabeledPoint] = MapPartitionsRDD[828] at map at <console>:53


In [13]:
parsedTestData.count()
parsedTrainData.count()

res5: Long = 277422


In [14]:
//Decision Tree
val numClasses = 2
val categoricalFeaturesInfo = Map[Int, Int]()
val impurity = "gini"
val maxDepth = 10
val maxBins = 32
val model_DT = DecisionTree.trainClassifier(parsedTrainData, numClasses, categoricalFeaturesInfo,impurity, maxDepth, maxBins)
val predictions_DT = parsedTestData.map(p => (model_DT.predict(p.features), p.label))
    

numClasses: Int = 2
categoricalFeaturesInfo: scala.collection.immutable.Map[Int,Int] = Map()
impurity: String = gini
maxDepth: Int = 10
maxBins: Int = 32
model_DT: org.apache.spark.mllib.tree.model.DecisionTreeModel = DecisionTreeModel classifier of depth 10 with 1179 nodes
predictions_DT: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[869] at map at <console>:55


In [15]:
//Random Forest
val numClasses = 2
val categoricalFeaturesInfo = Map[Int, Int]()
val numTrees = 10 // Use more in practice.
val featureSubsetStrategy = "auto" // Let the algorithm choose.
val impurity = "gini"
val maxDepth = 10
val maxBins = 32
val model_RF = RandomForest.trainClassifier(parsedTrainData, numClasses, categoricalFeaturesInfo,
  numTrees, featureSubsetStrategy, impurity, maxDepth, maxBins)
val predictions_RF = parsedTestData.map(p => (model_RF.predict(p.features), p.label))

numClasses: Int = 2
categoricalFeaturesInfo: scala.collection.immutable.Map[Int,Int] = Map()
numTrees: Int = 10
featureSubsetStrategy: String = auto
impurity: String = gini
maxDepth: Int = 10
maxBins: Int = 32
model_RF: org.apache.spark.mllib.tree.model.RandomForestModel =
TreeEnsembleModel classifier with 10 trees

predictions_RF: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[910] at map at <console>:64


In [16]:
val boostingStrategy = BoostingStrategy.defaultParams("Classification")
boostingStrategy.numIterations = 15
boostingStrategy.treeStrategy.maxDepth = 10
boostingStrategy.treeStrategy.categoricalFeaturesInfo = Map[Int, Int]()
val model_GB = GradientBoostedTrees.train(parsedTrainData, boostingStrategy)
val predictions_GB = parsedTestData.map(p => (model_GB.predict(p.features), p.label))

boostingStrategy: org.apache.spark.mllib.tree.configuration.BoostingStrategy = BoostingStrategy(org.apache.spark.mllib.tree.configuration.Strategy@5467042d,org.apache.spark.mllib.tree.loss.LogLoss$@69d0bee9,15,0.1,0.001)
boostingStrategy.numIterations: Int = 15
boostingStrategy.treeStrategy.maxDepth: Int = 10
boostingStrategy.treeStrategy.categoricalFeaturesInfo: Map[Int,Int] = Map()
model_GB: org.apache.spark.mllib.tree.model.GradientBoostedTreesModel =
TreeEnsembleModel classifier with 15 trees

predictions_GB: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[1473] at map at <console>:53


In [133]:
var predictions_supervised = Array(predictions_DT,predictions_RF,predictions_GB)

var acc_sup_arr :  Array[Double] = Array()
var roc_sup_arr : Array[Array[Array[Double]]] = Array()
var auroc_sup_arr :  Array[Double] = Array()
var prec_sup_arr: Array[Double] = Array()
var recall_sup_arr: Array[Double] = Array()
var f1_sup_arr: Array[Double] = Array()
val name = Array("DT","RF","GB")
var count = 0
for(p <- predictions_supervised){
    val accuracy = 100.0 * p.filter(x => x._1 == x._2).count() / (parsedTestData.count())
    val positively_Predicted = p.filter(x => x._1 == x._2).count()
//     println("Total Positive Prediction Correctly = " + positively_Predicted)
//     println("Accuracy = " + accuracy)
    var a = (accuracy* 1000).round / 1000.toDouble
    acc_sup_arr :+= accuracy
    
    val metrics = new BinaryClassificationMetrics(p)
    val roc = metrics.roc.collect().map(t=>Array(t._1,t._2))
    roc_sup_arr :+= roc
    
    val auROC = metrics.areaUnderROC
    auroc_sup_arr :+= auROC
//     println("Area under ROC = " + auROC)
    
    // Precision by threshold
    val precision = metrics.precisionByThreshold
    val precHighest = precision.collect()
    prec_sup_arr :+= precHighest(0)._2
    var pre = (precHighest(0)._2* 1000).round / 1000.toDouble

    // Recall by threshold
    val recall = metrics.recallByThreshold
    val recallHighest = recall.collect()
    recall_sup_arr :+= recallHighest(0)._2
    var r = (recallHighest(0)._2* 1000).round / 1000.toDouble

    // F1-score
    val f1Score = metrics.fMeasureByThreshold
    val f1Highest = f1Score.collect()
    f1_sup_arr :+= f1Highest(0)._2
    var f1 = (f1Highest(0)._2* 1000).round / 1000.toDouble
    
    var n = name(count)
    count+=1
    
    
    println(s"$n & $a & $pre & $r & $f1")
}

DT & 79.446 & 0.867 & 0.744 & 0.801
RF & 79.784 & 0.857 & 0.763 & 0.807
GB & 79.703 & 0.864 & 0.753 & 0.805


predictions_supervised: Array[org.apache.spark.rdd.RDD[(Double, Double)]] = Array(MapPartitionsRDD[869] at map at <console>:55, MapPartitionsRDD[910] at map at <console>:64, MapPartitionsRDD[1473] at map at <console>:53)
acc_sup_arr: Array[Double] = Array(79.44565717764809, 79.78423248838004, 79.70330110927964)
roc_sup_arr: Array[Array[Array[Double]]] = Array(Array(Array(0.0, 0.0), Array(0.1428929179605329, 0.7442768506632434), Array(1.0, 1.0), Array(1.0, 1.0)), Array(Array(0.0, 0.0), Array(0.15865304350729573, 0.762997432605905), Array(1.0, 1.0), Array(1.0, 1.0)), Array(Array(0.0, 0.0), Array(0.14823533340011352, 0.7531958707744972), Array(1.0, 1.0), Array(1.0, 1.0)))
auroc_sup_arr: Array[Double] = Array(0.8006919663513553, 0.8021721945493047, 0.8024802686871918)
prec_sup_arr: Array[Do...


In [112]:
//Unsupervised
val predictions_CN = parsedTestData.map(p => (p.features(2), p.label))
val predictions_JC = parsedTestData.map(p => (p.features(3), p.label))
val predictions_AA = parsedTestData.map(p => (p.features(4), p.label))
val predictions_RA = parsedTestData.map(p => (p.features(5), p.label))
val predictions_PA = parsedTestData.map(p => (p.features(6), p.label))
val predictions_AR = parsedTestData.map(p => (p.features(7), p.label))
val predictions_ND = parsedTestData.map(p => (p.features(8), p.label))
val predictions_TN = parsedTestData.map(p => (p.features(9), p.label))
val predictions_UND = parsedTestData.map(p => (p.features(10), p.label))
val predictions_VND = parsedTestData.map(p => (p.features(11), p.label))
val predictions_SC = parsedTestData.map(p => (p.features(12), p.label))

// var predictions_unsupervised = Array(predictions_CN)
var predictions_unsupervised = Array(predictions_CN,predictions_JC,predictions_AA,predictions_RA,predictions_PA,predictions_AR,predictions_ND,predictions_TN,predictions_UND,predictions_VND,predictions_SC)

predictions_CN: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[176425] at map at <console>:60
predictions_JC: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[176426] at map at <console>:61
predictions_AA: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[176427] at map at <console>:62
predictions_RA: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[176428] at map at <console>:63
predictions_PA: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[176429] at map at <console>:64
predictions_AR: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[176430] at map at <console>:65
predictions_ND: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[176431] at map at <console>:66
predictions_TN: org.apache.spark.r...


In [88]:
var roc_unsup_arr : Array[Array[Array[Double]]] = Array()
var auroc_unsup_arr :  Array[Double] = Array()

for(p <- predictions_unsupervised){
    val metrics = new BinaryClassificationMetrics(p)
    val roc = metrics.roc.collect().map(t=>Array(t._1,t._2))
    roc_unsup_arr :+= roc
    val auROC = metrics.areaUnderROC
    auroc_unsup_arr :+= auROC
    println("Area under ROC = " + auROC)
     val truePos = p.filter{case (a,b) => b==1}.count()
    val trueNeg = p.filter{case (a,b) => b==0}.count()
    
    val predArr = p.collect()
    val precision = metrics.precisionByThreshold
    
    val thresholds = precision.map(_._1).collect()
    
    val tpr_fpr = thresholds.map(t => {
      var tp = 0;
      var fp = 0;
//       println(t)
        
      predArr.map(r => {

        if (r._2 == 1 && r._1 > t){
            tp=tp+1;

        } // true positive
        if (r._2 == 0 && r._1 > t){
            fp=fp+1;
        }
      })

      (t,tp.toFloat/truePos-fp.toFloat/trueNeg)
    })
    
    println(tpr_fpr.maxBy(_._2))

    
    //     metrics.roc.collect.foreach { x =>
//       println(x)
//     }
    
    
    
//     // Precision by threshold
//     val precision = metrics.precisionByThreshold
//     precision.collect.foreach { case (t, p) =>
//       println(s"Threshold: $t, Precision: $p")
//     }
//     // Recall by threshold
//     val recall = metrics.recallByThreshold
//     recall.collect.foreach { case (t, r) =>
//       println(s"Threshold: $t, Recall: $r")
//     }
//     val joinRDD = precision.join(recall).map{case (a,(c,d))=>(a,c-d)}
// //     println(joinRDD)
//     joinRDD.collect.foreach { t =>
//       println(t)
//     }
//     println(joinRDD.maxBy(_._2))
//     joinRDD.reduce((a,b)=> a._1 + b._1)
    // Compute thresholds used in ROC and PR curves
//     val max
//     val thresholds = precision.map(_._1)
//     thresholds.filter{thr=>joinRDD}
//     thresholds.collect.foreach { thr =>
//         var p = precision.filter{case (t, p) => t==thr}
//         var r = recall.filter{case (t, p) => t==thr}
        
//     }

//     inputRDD.reduce( )
//     (a,b)=> ("max",a._2 min b._2)
    
    


//     // F-measure
//     val f1Score = metrics.fMeasureByThreshold
// //     f1Score.collect.foreach { case (t, f) =>
// //       println(s"Threshold: $t, F-score: $f, Beta = 1")
// //     }
//     println(f1Score.collect.maxBy(_._2))
}

//Pa 4, tn 7, und 8, vnd 9, sc 10

Area under ROC = 0.7358160137879454
(1.0,0.408999)
Area under ROC = 0.7467962074268947
(0.043795620437956206,0.44389284)
Area under ROC = 0.7561505153585301
(1.1061923157604907,0.48860496)
Area under ROC = 0.757853099189961
(0.08382352941176471,0.49218893)
Area under ROC = 0.4548192595246393
(2472.0,0.0221174)
Area under ROC = 0.7935376164565342
(0.09089423397820755,0.44413823)
Area under ROC = 0.7459152291931872
(0.13608276348795434,0.45470116)
Area under ROC = 0.42353003830570585
(166.0,0.018414682)
Area under ROC = 0.44088407415788267
(0.0,3.579259E-4)
Area under ROC = 0.4791516427082723
(74.0,0.0071356315)
Area under ROC = 0.48382696133276804
(1.0,0.0)


roc_unsup_arr: Array[Array[Array[Double]]] = Array(Array(Array(0.0, 0.0), Array(0.0, 2.6743688489516474E-5), Array(0.0, 4.011553273427471E-5), Array(0.0, 6.685922122379119E-5), Array(0.0, 8.023106546854942E-5), Array(0.0, 1.2034659820282414E-4), Array(0.0, 1.6046213093709883E-4), Array(0.0, 1.8720581942661531E-4), Array(0.0, 2.0057766367137357E-4), Array(0.0, 2.139495079161318E-4), Array(0.0, 2.5406504065040653E-4), Array(0.0, 2.941805733846812E-4), Array(0.0, 3.7441163885323063E-4), Array(0.0, 4.1452717158750536E-4), Array(0.0, 4.278990158322636E-4), Array(0.0, 4.6801454856653827E-4), Array(0.0, 4.947582370560548E-4), Array(0.0, 5.081300813008131E-4), Array(0.0, 5.482456140350877E-4), Array(0.0, 5.749893025246041E-4), Array(0.0, 6.017329910141207E-4), Array(0.0, 6.284766795036372E-4), ...


In [113]:
// CN=1.0,JC=0.043795620437956206,AA=1.1061923157604907,RA=,PA,AR,ND,TN,UND,VND,SC
val bestT = Array(1.0,0.043795620437956206,1.1061923157604907,0.08382352941176471,2472.0,0.09089423397820755,0.13608276348795434,166.0,0.0,74.0,1.0)

bestT: Array[Double] = Array(1.0, 0.043795620437956206, 1.1061923157604907, 0.08382352941176471, 2472.0, 0.09089423397820755, 0.13608276348795434, 166.0, 0.0, 74.0, 1.0)


In [125]:
val name = Array("CN","JC","AA","RA","PA","AR","ND","TN","UND","VND","SC")
var count = 0
for(p <- predictions_unsupervised){
    val metrics = new BinaryClassificationMetrics(p)
    val roc = metrics.roc.collect().map(t=>Array(t._1,t._2))
    roc_unsup_arr :+= roc
    val auROC = metrics.areaUnderROC
    auroc_unsup_arr :+= auROC
//     println("Area under ROC = " + auROC)
   
    val best = bestT(count)
    
    // Precision by threshold
    val precision = metrics.precisionByThreshold
    val preFilter = precision.filter{case(t,r)=>t==best}.collect()
    val pre = (preFilter(0)._2* 1000).round / 1000.toDouble
//     val pre = preFilter(0)._2
//     val pre = 1
    
    // Recall by threshold
    val recall = metrics.recallByThreshold
    val reFilter = recall.filter{case(t,r)=>t==best}.collect()
    val r = (reFilter(0)._2* 1000).round / 1000.toDouble
//     val r = reFilter(0)._2
//     val r = 1
    //F-measure
    val f1Score = metrics.fMeasureByThreshold
    val f1Filter = f1Score.filter{case(t,r)=>t==best}.collect()
    val f = (f1Filter(0)._2* 1000).round / 1000.toDouble
//     val f = f1Filter(0)._2
//     val f = 1
    //Accuracy
    var a = 100.0 * p.filter(x => {
        val predicted = if(x._1>best) 1 else 0
         predicted == x._2
    }).count() / (parsedTestData.count())
    a = (a* 1000).round / 1000.toDouble
    
    val n = name(count)
//     println(s"Threshold: $best, Accuracy: $a, Precision: $pre, Recall: $r, F1-Score: $f")
    println(s"$n & $a & $pre & $r & $f & $best")
//     val joinRDD = precision.join(recall).map{case (a,(c,d))=>(a,c-d)}
// //     println(joinRDD)
//     joinRDD.collect.foreach { t =>
//       println(t)
//     }
//     println(joinRDD.maxBy(_._2))
//     joinRDD.reduce((a,b)=> a._1 + b._1)
    // Compute thresholds used in ROC and PR curves
//     val max
//     val thresholds = precision.map(_._1)
//     thresholds.filter{thr=>joinRDD}
//     thresholds.collect.foreach { thr =>
//         var p = precision.filter{case (t, p) => t==thr}
//         var r = recall.filter{case (t, p) => t==thr}
        
//     }

//     inputRDD.reduce( )
//     (a,b)=> ("max",a._2 min b._2)
    
 count=count+1

//     println(f1Score.collect.maxBy(_._2))
}

CN & 67.756 & 0.723 & 0.664 & 0.693 & 1.0
JC & 70.505 & 0.85 & 0.569 & 0.682 & 0.043795620437956206
AA & 72.364 & 0.91 & 0.557 & 0.691 & 1.1061923157604907
RA & 72.503 & 0.916 & 0.556 & 0.692 & 0.08382352941176471
PA & 45.893 & 0.74 & 0.039 & 0.075 & 2472.0
AR & 70.469 & 0.853 & 0.565 & 0.68 & 0.09089423397820755
ND & 70.986 & 0.859 & 0.571 & 0.686 & 0.13608276348795434
TN & 45.574 & 0.814 & 0.026 & 0.05 & 166.0
UND & 55.267 & 0.555 & 1.0 & 0.714 & 0.0
VND & 45.074 & 0.631 & 0.026 & 0.05 & 74.0
SC & 44.474 & 0.546 & 0.844 & 0.663 & 1.0


name: Array[String] = Array(CN, JC, AA, RA, PA, AR, ND, TN, UND, VND, SC)
count: Int = 11


In [38]:
//Convert to Dataframe and ML Pipeline to use Feature Importance
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.classification.RandomForestClassifier
import org.apache.spark.ml.classification.RandomForestClassificationModel
import org.apache.spark.ml.feature.{StringIndexer, IndexToString, VectorIndexer}
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.mllib.util.MLUtils


val testing = testFeature.toDF("n1","n2","label","CN","JC","AA","RA","PA","AR","ND","TN","UND","VND","SC").cache()
val training = trainFeature.toDF("n1","n2","label","CN","JC","AA","RA","PA","AR","ND","TN","UND","VND","SC").cache()

val assembler = new VectorAssembler()
  .setInputCols(Array("CN","JC","AA","RA","PA","AR","ND","TN","UND","VND","SC"))
  .setOutputCol("features")

val trainingData = assembler.setHandleInvalid("keep").transform(training)

val labelIndexer = new StringIndexer()
  .setInputCol("label")
  .setOutputCol("indexedLabel")
  .fit(trainingData)
// // Automatically identify categorical features, and index them.
// // Set maxCategories so features with > 4 distinct values are treated as continuous.

val featureIndexer = new VectorIndexer()
  .setInputCol("features")
  .setOutputCol("indexedFeatures")
  .fit(trainingData)

// Train a RandomForest model.
val rf = new RandomForestClassifier()
  .setLabelCol("indexedLabel")
  .setFeaturesCol("indexedFeatures")
  .setNumTrees(10)
  .setMaxDepth(10)


// Convert indexed labels back to original labels.
val labelConverter = new IndexToString()
  .setInputCol("prediction")
  .setOutputCol("predictedLabel")
  .setLabels(labelIndexer.labels)

// Chain indexers and forest in a Pipeline
val pipeline = new Pipeline()
  .setStages(Array(labelIndexer, featureIndexer, rf, labelConverter))

// Train model.  This also runs the indexers.
val model = pipeline.fit(trainingData)

val testingData = assembler.setHandleInvalid("keep").transform(testing)
// Make predictions.
val predictions = model.transform(testingData)

// Select example rows to display.
predictions.select("predictedLabel", "label", "features").show(5)

// Select (prediction, true label) and compute test error
val evaluator = new MulticlassClassificationEvaluator()
  .setLabelCol("indexedLabel")
  .setPredictionCol("prediction")
  .setMetricName("accuracy")

val accuracy = evaluator.evaluate(predictions)
println(accuracy)

val rfModel = model.stages(2).asInstanceOf[RandomForestClassificationModel]
// println("Learned classification forest model:\n" + rfModel.toDebugString)

rfModel.featureImportances

+--------------+-----+--------------------+
|predictedLabel|label|            features|
+--------------+-----+--------------------+
|             1|    1|[3.0,0.25,2.96759...|
|             1|    1|[1.0,0.2,0.756304...|
|             0|    1|[1.0,0.0256410256...|
|             1|    1|[5.0,0.3846153846...|
|             1|    1|[28.0,0.345679012...|
+--------------+-----+--------------------+
only showing top 5 rows

0.7787848221437683


import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.classification.RandomForestClassifier
import org.apache.spark.ml.classification.RandomForestClassificationModel
import org.apache.spark.ml.feature.{StringIndexer, IndexToString, VectorIndexer}
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.mllib.util.MLUtils
testing: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [n1: bigint, n2: bigint ... 12 more fields]
training: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [n1: bigint, n2: bigint ... 12 more fields]
assembler: org.apache.spark.ml.feature.VectorAssembler = VectorAssembler: uid=vecAssembler_84cb5fc752ae, handleI...


In [40]:
rfModel.featureImportances

res19: org.apache.spark.ml.linalg.Vector = (11,[0,1,2,3,4,5,6,7,8,9,10],[0.00621919918917448,0.06070710416989457,0.2811441020795655,0.35530048229073624,0.06342308680438571,0.06666146882098178,0.006187987373674058,0.05658488837683489,0.02008176229570667,0.07552673892840452,0.008163179670641412])


In [70]:
//Record output
var output=""
output+=acc_sup_arr.mkString(",")
output+="\n"
output+=roc_sup_arr.flatten.flatten.mkString(",")
output+="\n"
output+=auroc_sup_arr.mkString(",")
output+="\n"
var a = 0;
for(a <- 0 until 11){
    output+=roc_unsup_arr(a).flatten.mkString(",")
    output+="\n"
}
output+=auroc_unsup_arr.mkString(",")
output+="\n"
output+=rfModel.featureImportances.toString

Files.write(Paths.get("link_pred_output.txt"), output.getBytes(StandardCharsets.UTF_8))

output: String =
79.86990832764708,80.01082861126768,80.11614798113152
0.0,0.0,0.1497044976459982,0.757453623381823,1.0,1.0,1.0,1.0,0.0,0.0,0.1574343049851414,0.7661684238622715,1.0,1.0,1.0,1.0,0.0,0.0,0.14998831346622593,0.7621113038836247,1.0,1.0,1.0,1.0
0.8038745628679124,0.804367059438565,0.8060614952086994
0.0,0.0,0.0,1.3345789403443214E-5,0.0,2.669157880688643E-5,0.0,4.003736821032964E-5,0.0,5.338315761377286E-5,0.0,8.007473642065928E-5,0.0,1.0676631522754571E-4,0.0,1.2011210463098892E-4,0.0,1.3345789403443215E-4,0.0,1.8684105164820498E-4,0.0,2.001868410516482E-4,0.0,2.1353263045509143E-4,0.0,2.669157880688643E-4,0.0,2.936073668757507E-4,0.0,3.2029894568263713E-4,0.0,3.3364473508608035E-4,0.0,3.603363138929668E-4,0.0,3.7368210329640996E-4,0.0,3.870278926998532E-4,0.0,4.40411050313...
